### DETAILED EXPLANATION

This script generates blurred versions of each image in the dataset using three types of blur:

**Why Blurring Images?**

Blurring is a common type of image distortion that can occur due to various factors:

- Camera motion during capture (Motion Blur)
- Out-of-focus regions due to depth of field (Gaussian Blur)
- Uniform blurring due to poor image processing or resizing (Box Blur)

These blurs simulate real-world distortions, making our analysis more realistic.

**Types of Blur Used**

1. Gaussian Blur:
   - This blur applies a Gaussian kernel to each pixel, which results in a smooth, out-of-focus effect.
   - The blur is strongest at the center and gradually fades out (bell curve distribution).
   - We chose sigma range (0.5 to 4.0) because it provides visible but controlled blur.
   - Sigma defines the standard deviation of the Gaussian distribution (higher = stronger blur).

2. Motion Blur:
   - Simulates the effect of movement, either from a moving object or a moving camera.
   - The blur has a clear direction (angle) and intensity (length).
   - We chose length (3 to 30) to allow visible motion but not too extreme.
   - Angle (0 to 360) provides all possible directions of movement.

3. Box Blur:
   - Applies a simple averaging of pixel values in a square region around each pixel.
   - This blur is fast and commonly used for quick blur effects.
   - We chose kernel size (3 to 15) because it provides a visible blur without being too extreme.
   - The kernel size defines the square region used for averaging (higher = stronger blur).

**Why Save Parameters in a Parquet File?**

- Parquet is optimized for speed and storage efficiency.
- It is suitable for large datasets, ensuring fast read and write operations.
- Parquet allows us to store the blur parameters alongside the image identifiers, making it easy to track and analyze.

In [1]:
import numpy as np
import cv2

def gen_gaussian_kernel(size, sigma):
    """Generate a Gaussian blur kernel."""
    if size % 2 == 0:
        size = size + 1
    k1d = cv2.getGaussianKernel(size, sigma)
    kernel = k1d @ k1d.T
    return kernel

def gen_motion_kernel(length, angle):
    """Generate a motion blur kernel based on length and angle."""
    if length % 2 == 0:
        length += 1

    kernel = np.zeros((length, length), dtype=np.float32)
    center = (length - 1) / 2.0

    angle_rad = np.deg2rad(angle)
    dx = length / 2.0 * np.cos(angle_rad)
    dy = length / 2.0 * np.sin(angle_rad)

    start_point = (int(center - dx), int(center - dy))
    end_point = (int(center + dx), int(center + dy))
    
    cv2.line(kernel, start_point, end_point, 1, 1, lineType=cv2.LINE_AA)
    kernel /= np.sum(kernel)
    return kernel

def gen_box_kernel(size):
    """Generate a box blur kernel."""
    return np.ones((size, size), dtype=np.float32) / (size * size)

In [ ]:
import cv2
import pandas as pd
import random
from tqdm import tqdm

from common.config import TEST_ORIGINAL_DIR, TEST_BLURRED_DIR, BLUR_PARAM_RANGES

# Sample parameters: None for whole dataset
SAMPLE_SIZE = None

image_files = sorted(list((TEST_ORIGINAL_DIR / "00000").glob("*.png")))

if SAMPLE_SIZE is not None:
    image_files = image_files[:SAMPLE_SIZE]

for blur_type in BLUR_PARAM_RANGES.keys():
    (TEST_BLURRED_DIR / blur_type).mkdir(parents=True, exist_ok=True)

blurred_images = []

for img_path in tqdm(image_files):
    image = cv2.imread(str(img_path))
    if image is None:
        continue
    original_name = img_path.name

    blur_params = {
        "key": original_name.replace('.png', '')
    }

    for blur_type, params in BLUR_PARAM_RANGES.items():
        gen_kernel = None
        match blur_type:
            case "box":
                gen_kernel = gen_box_kernel
            case "gaussian":
                gen_kernel = gen_gaussian_kernel
            case "motion":
                gen_kernel = gen_motion_kernel
            case _:
                raise ValueError(f"gen_kernel was not defined for {blur_type}")

        args = {param: val_type(random.uniform(val_min, val_max)) for param, (val_min, val_max, val_type) in params.items()}
        kernel = gen_kernel(**args)
        blurred = cv2.filter2D(image, -1, kernel)

        for param, val in args.items():
            blur_params[f"{blur_type}_{param}"] = val
        
        cv2.imwrite(str(TEST_BLURRED_DIR / blur_type / original_name), blurred)
    
    blurred_images.append(blur_params)

df_blur = pd.DataFrame(blurred_images)
print("Blur generation completed.")

100%|██████████| 950/950 [05:45<00:00,  2.75it/s]

Blur generation completed.


In [3]:
df_blur

,key,box_size,gaussian_size,gaussian_sigma,motion_length,motion_angle
0,000000000,7,5,3.583921,22,354.158595
1,000000002,6,8,2.132541,28,290.625236
2,000000003,8,6,1.306195,11,4.899292
3,000000004,12,6,3.674510,20,20.763211
4,000000005,6,18,1.017260,28,326.191100
...,...,...,...,...,...,...
945,000001245,8,8,3.533695,5,59.677990
946,000001246,5,9,0.726179,17,169.055718
947,000001247,9,17,3.767770,26,351.008914
948,000001248,10,11,3.551345,25,95.880235


In [4]:
import pandas as pd
from common.config import IDPA_DATASET

df_original = pd.read_parquet(IDPA_DATASET)
df_original

,url,category,key,width,height,exif,aspect_ratio,size,rms_contrast,sobel_edge_strength,canny_edge_density
0,http://100500foto.com/wp-content/uploads/2016/...,people,000000291,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
1,http://2gfsl7am0og1m91u0pwpiehl.wpengine.netdn...,indoor_scene,000000987,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
2,http://411posters.com/wp-content/uploads/2011/...,poster,000000382,1300.0,1728.0,{},0.752315,3169036.0,0.373365,54.414061,0.139681
3,http://RealEstateAdminImages.gabriels.net/170/...,architecture,000000058,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
4,http://RealEstateAdminImages.gabriels.net/170/...,architecture,000000001,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1245,https://www.yamaha.com/en/musical_instrument_g...,complex,000000564,1800.0,1042.0,{},1.727447,3199285.0,0.331964,60.298111,0.093565
1246,https://www.yellowmaps.com/usgs/topomaps/drg24...,map,000001225,1509.0,2026.0,"{""Image Tag 0x5100"": ""0""}",0.744817,5742001.0,0.221090,99.311477,0.177919
1247,https://www.zappos.com/images/z/2/5/1/8/8/7/25...,furniture,000000481,1920.0,1440.0,{},1.333333,4276346.0,0.253243,53.379880,0.086103
1248,https://ycdn.space/h/2015/02/Capitol-Hill-Loft...,indoor_scene,000000954,1050.0,1575.0,{},0.666667,2000861.0,0.241754,64.152358,0.085650


In [5]:
# Merge LEFT per mantenere tutte le righe di df_original
df_merged = pd.merge(
    df_original,
    df_blur,
    on='key',
    how='left'
)
df_merged

,url,category,key,width,height,exif,aspect_ratio,size,rms_contrast,sobel_edge_strength,canny_edge_density,box_size,gaussian_size,gaussian_sigma,motion_length,motion_angle
0,http://100500foto.com/wp-content/uploads/2016/...,people,000000291,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://2gfsl7am0og1m91u0pwpiehl.wpengine.netdn...,indoor_scene,000000987,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://411posters.com/wp-content/uploads/2011/...,poster,000000382,1300.0,1728.0,{},0.752315,3169036.0,0.373365,54.414061,0.139681,7.0,6.0,3.524562,14.0,335.353107
3,http://RealEstateAdminImages.gabriels.net/170/...,architecture,000000058,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://RealEstateAdminImages.gabriels.net/170/...,architecture,000000001,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,https://www.yamaha.com/en/musical_instrument_g...,complex,000000564,1800.0,1042.0,{},1.727447,3199285.0,0.331964,60.298111,0.093565,13.0,7.0,2.560898,9.0,155.191431
1246,https://www.yellowmaps.com/usgs/topomaps/drg24...,map,000001225,1509.0,2026.0,"{""Image Tag 0x5100"": ""0""}",0.744817,5742001.0,0.221090,99.311477,0.177919,13.0,5.0,3.746691,8.0,173.703361
1247,https://www.zappos.com/images/z/2/5/1/8/8/7/25...,furniture,000000481,1920.0,1440.0,{},1.333333,4276346.0,0.253243,53.379880,0.086103,8.0,15.0,3.466276,20.0,78.233455
1248,https://ycdn.space/h/2015/02/Capitol-Hill-Loft...,indoor_scene,000000954,1050.0,1575.0,{},0.666667,2000861.0,0.241754,64.152358,0.085650,7.0,9.0,1.854978,19.0,80.733701


In [6]:
from common.config import IDPA_DATASET

df_merged.to_parquet(IDPA_DATASET)